# Analyse der PV-Anlagen in der Schweiz
Dieses Notebook analysiert die Photovoltaik-Anlagen (PV-Anlagen) in der Schweiz, einschließlich ihrer Verteilung und Entwicklung.

## Daten laden, aufbereiten und anreichern
In diesem Abschnitt werden die Daten geladen, gefiltert und für die Analyse vorbereitet.

In [ ]:
# Bibliotheken importieren
import pandas as pd

# CSV-Datei mit den PV-Anlagen laden
file_path = '../data/electricity_production_plants/ElectricityProductionPlant.csv'
df = pd.read_csv(file_path)

# Kategorien ersetzen (Haupt-, Unter- und Anlagenkategorien)
main_categories = pd.read_csv('../data/electricity_production_plants/MainCategoryCatalogue.csv')
df['MainCategory'] = df['MainCategory'].replace(dict(zip(main_categories['Catalogue_id'], main_categories['de'])))

sub_categories = pd.read_csv('../data/electricity_production_plants/SubCategoryCatalogue.csv')
df['SubCategory'] = df['SubCategory'].replace(dict(zip(sub_categories['Catalogue_id'], sub_categories['de'])))

plant_categories = pd.read_csv('../data/electricity_production_plants/PlantCategoryCatalogue.csv')
df['PlantCategory'] = df['PlantCategory'].replace(dict(zip(plant_categories['Catalogue_id'], plant_categories['de'])))

# Nur Photovoltaik-Anlagen filtern
df = df[df['SubCategory'] == 'Photovoltaik'].copy()

# Jahr aus 'BeginningOfOperation' extrahieren
df['Year'] = pd.to_datetime(df['BeginningOfOperation'], errors='coerce').dt.year

# Erste Zeilen anzeigen
df.head()

## Karte mit allen PV-Anlagen in der Schweiz (Stand 2024)
Diese Karte zeigt die Standorte aller PV-Anlagen in der Schweiz bis zum Jahr 2024.

In [ ]:
# Bibliotheken für die Kartenerstellung importieren
import folium
from folium.plugins import FastMarkerCluster
from pyproj import Transformer

# Daten für Installationen bis 2024 filtern
df['BeginningOfOperation'] = pd.to_datetime(df['BeginningOfOperation'], errors='coerce')
df = df[df['BeginningOfOperation'].dt.year <= 2024]

# Karte erstellen
map = folium.Map(location=[46.8, 8.33], zoom_start=7, tiles='OpenStreetMap')

# Marker-Cluster hinzufügen
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)
coordinates = [
    [transformer.transform(row['_x'], row['_y'])[1], transformer.transform(row['_x'], row['_y'])[0]]
    for _, row in df.iterrows() if not pd.isna(row['_x']) and not pd.isna(row['_y'])
]
FastMarkerCluster(coordinates).add_to(map)

# Titel hinzufügen
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>PV-Anlagen in der Schweiz (Stand 2024)</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

# Karte speichern und anzeigen
map.save('../docs/assets/diagramme/elcoms_map.html')
map

## Kumulative Entwicklung der Gesamtkapazität ab 2015
Dieses Diagramm zeigt die kumulative Entwicklung der Gesamtkapazität der PV-Anlagen pro Kanton ab dem Jahr 2015.

In [ ]:
# Plotly für die Visualisierung importieren
import plotly.express as px

# Daten gruppieren und kumulativ berechnen
cumulative_data = df.groupby(['Year', 'Canton'])['TotalPower'].sum().groupby(level=1).cumsum().reset_index()
filtered_data = cumulative_data[(cumulative_data['Year'] >= 2015) & (cumulative_data['Year'] <= 2024)].copy()
filtered_data['CumulativePower'] = filtered_data.groupby('Canton')['TotalPower'].cumsum()

# Farben für die Top 4 Kantone und neutrale Farbe für die restlichen
top_4_cantons = ['BE', 'ZH', 'VD', 'SG']
filtered_data['Color'] = filtered_data['Canton'].apply(
    lambda x: (
        'rgb(0, 102, 204)' if x == top_4_cantons[0] else
        'rgb(0, 153, 76)' if x == top_4_cantons[1] else
        'rgb(255, 153, 51)' if x == top_4_cantons[2] else
        'rgb(204, 0, 0)' if x == top_4_cantons[3] else
        'rgb(169, 169, 169)'
    )
)

# Liniendiagramm erstellen
fig = px.line(
    filtered_data,
    x='Year',
    y='CumulativePower',
    color='Canton',
    title='Kumulative Entwicklung der Gesamtkapazität pro Kanton ab 2015 (MW)',
    labels={'Year': 'Jahr', 'CumulativePower': 'Kumulative Gesamtkapazität (MW)', 'Canton': 'Kanton'},
    color_discrete_map={canton: color for canton, color in zip(filtered_data['Canton'], filtered_data['Color'])}
)
fig.update_traces(
    line=dict(width=2),
    selector=dict(name=lambda name: name not in top_4_cantons),
    line_color='rgb(169, 169, 169)'
)
fig.update_layout(
    xaxis=dict(tickmode='linear', tick0=2015, dtick=1),
    template='simple_white',
    legend_title_text='Kanton'
)
fig.show()
fig.write_html("../docs/assets/diagramme/elcom_entwicklung_total_power.html")

## Gesamtkapazität pro Kanton im Jahr 2024
Dieses Balkendiagramm zeigt die Gesamtkapazität der PV-Anlagen pro Kanton im Jahr 2024.

In [ ]:
# Daten für das Jahr 2024 filtern
data_2024 = df[df['Year'] == 2024].groupby('Canton')['TotalPower'].sum().reset_index()

# Top 4 Kantone nach Gesamtkapazität identifizieren
data_2024 = data_2024.sort_values(by='TotalPower', ascending=False)
top_4_cantons = data_2024.head(4)['Canton'].tolist()

# Farben für die Top 4 Kantone und neutrale Farbe für die restlichen
data_2024['Color'] = data_2024['Canton'].apply(
    lambda x: (
        'rgb(0, 102, 204)' if x == top_4_cantons[0] else
        'rgb(0, 153, 76)' if x == top_4_cantons[1] else
        'rgb(255, 153, 51)' if x == top_4_cantons[2] else
        'rgb(204, 0, 0)' if x == top_4_cantons[3] else
        'rgb(169, 169, 169)'
    )
)

# Balkendiagramm erstellen
fig = px.bar(
    data_2024,
    x='Canton',
    y='TotalPower',
    title='Gesamtkapazität pro Kanton im Jahr 2024 (MW)',
    labels={'Canton': 'Kanton', 'TotalPower': 'Gesamtkapazität (MW)'},
    text='TotalPower',
    color='Color',
    color_discrete_map='identity'
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    xaxis=dict(categoryorder='total descending'),
    template='simple_white',
    legend_title_text='Kanton',
    showlegend=False
)
fig.show()
fig.write_html("../docs/assets/diagramme/elcom_total_power_2024.html")

## Karte mit kumulierter Gesamtkapazität der PV-Anlagen pro Gemeinde bis 2024
Diese Karte zeigt die kumulierte Gesamtkapazität der PV-Anlagen in jeder Gemeinde bis zum Jahr 2024.

In [ ]:
import locale
import folium
import json
from shapely.geometry import shape, mapping
from shapely.ops import transform
from functools import partial
import pyproj

# Load CSV File
file_path = '../data/electricity_production_plants/AMTOVZ_CSV_WGS84.csv'
df_bfs_to_plz = pd.read_csv(file_path, delimiter=';')

# Merge and identify non-matching rows
merged_df = df.merge(
    df_bfs_to_plz[['PLZ', 'BFS-Nr']],
    left_on='PostCode',
    right_on='PLZ',
    how='left',
    indicator=True
)

# Prepare cumulative data
cumulative_map_data = merged_df[merged_df['Year'] <= 2024].groupby(['BFS-Nr'])['TotalPower'].sum().reset_index()

# Load and simplify GeoJSON
with open('../data/electricity_production_plants/gemeinden.geojson', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Assign default values to missing municipalities
average_power = cumulative_map_data['TotalPower'].mean()
missing_bfs = set(geojson_data['features'][i]['properties']['gemeinde_BFS_NUMMER'] for i in range(len(geojson_data['features']))) - set(cumulative_map_data['BFS-Nr'])
for bfs in missing_bfs:
    cumulative_map_data = pd.concat([cumulative_map_data, pd.DataFrame({'BFS-Nr': [bfs], 'TotalPower': [average_power]})], ignore_index=True)

simplified_features = []
for feature in geojson_data['features']:
    geom = shape(feature['geometry'])
    simplify = partial(
        pyproj.Transformer.from_crs('EPSG:4326', 'EPSG:4326', always_xy=True).transform
    )
    simplified_geom = transform(simplify, geom).simplify(0.001, preserve_topology=True)
    feature['geometry'] = mapping(simplified_geom)
    bfs_nr = feature['properties']['gemeinde_BFS_NUMMER']
    total_power = cumulative_map_data.loc[cumulative_map_data['BFS-Nr'] == bfs_nr, 'TotalPower'].values[0] if bfs_nr in cumulative_map_data['BFS-Nr'].values else 0
    feature['properties']['TotalPower'] = f"{int(total_power):,}".replace(",", "'")
    feature['properties']['bfs-str'] = str(feature['properties']['gemeinde_BFS_NUMMER'])
    simplified_features.append(feature)

geojson_data['features'] = simplified_features

map = folium.Map(location=[46.8, 8.33], zoom_start=7)

# Add choropleth layer
folium.Choropleth(
    geo_data=geojson_data,
    name='Choropleth',
    data=cumulative_map_data,
    columns=['BFS-Nr', 'TotalPower'],
    key_on='feature.properties.gemeinde_BFS_NUMMER',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Total Power (MW)'
).add_to(map)

# Add tooltips and customize boundary style
folium.GeoJson(
    geojson_data,
    name="Gemeinden",
    style_function=lambda feature: {
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['gemeinde_NAME', 'bfs-str', 'TotalPower'],
        aliases=['Gemeinde:', 'BFS-Nummer:', 'Gesamtkapazität (MW):'],
        localize=True
    )
).add_to(map)

# Add title and save map
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>Heatmap von der Leistung der PV Anlagen pro Gemeinde</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

map.save('../docs/assets/diagramme/map_cumulative_pv_power.html')

map